In [24]:
import os
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow
import boto3
from botocore.exceptions import ClientError
from datetime import datetime as dt

In [25]:
sagemaker_session = sagemaker.Session()

role = "role_sagemaker"
region = sagemaker_session.boto_session.region_name

In [26]:
data_bucket = "health-insurance-cross-sell"
data_uri = "s3://"+data_bucket

## Caricare i dati su S3

In [31]:
today = dt.now()
folder = today.strftime("%Y-%m-%d")

s3 = boto3.client('s3')

with open("../app/app/predictions/data.csv", "rb") as f:
    s3.upload_fileobj(f, data_bucket, folder+"/data.csv")

with open("../app/app/model/model.h5", "rb") as f:
    s3.upload_fileobj(f, data_bucket, "model.h5")

training_data_uri = data_uri+"/"+folder

## Addestramento

In [32]:
estimator = TensorFlow(
    entry_point="training_script.py",
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    framework_version="2.1.0",
    py_version="py3",
    #model_dir=training_data_uri
    #wait=False
)

In [33]:
estimator.fit(data_uri)

2021-06-03 15:41:26 Starting - Starting the training job...
2021-06-03 15:41:49 Starting - Launching requested ML instancesProfilerReport-1622734882: InProgress
......
2021-06-03 15:42:49 Starting - Preparing the instances for training......
2021-06-03 15:44:10 Downloading - Downloading input data...
2021-06-03 15:44:44 Training - Training image download completed. Training in progress..2021-06-03 15:44:49,820 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2021-06-03 15:44:49,828 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-06-03 15:44:50,194 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-06-03 15:44:50,210 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-06-03 15:44:50,226 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-06-03 15:44:50,236 sagemaker-containers INFO     Invoking user script

Training Env:

{


2021-06-03 15:44:55,438 sagemaker-containers INFO     Reporting training SUCCESS

2021-06-03 15:45:10 Uploading - Uploading generated training model
2021-06-03 15:45:10 Completed - Training job completed
Training seconds: 71
Billable seconds: 71


In [34]:
estimator.model_dir

's3://sagemaker-us-west-2-248207680755/tensorflow-training-2021-06-03-15-41-21-081/model'

## Inferenza

In [67]:
predictor = estimator.deploy(initial_instance_count=1, instance_type="ml.p2.xlarge")

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


---------------!

In [75]:
import pandas as pd
from training import _preprocess

df = pd.DataFrame(
    data=[[1,"Male",44,1,28.0,0,"> 2 Years","Yes",40454.0,26.0,217,1]], 
    columns=[
    "id","Gender","Age","Driving_License","Region_Code","Previously_Insured","Vehicle_Age","Vehicle_Damage",
    "Annual_Premium","Policy_Sales_Channel","Vintage","Response"
])

df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1


In [78]:
x, y = _preprocess(df, base_dir="model")
x.shape

C:\Users\gfgul\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\gfgul\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator ColumnTransformer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\Users\gfgul\Anaconda3\lib\site-packages\sklearn\base.py:315: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


(1, 216)

In [81]:
predictions = predictor.predict(x)
predictions

{'predictions': [[0.455579937]]}

## Rimuovere l'endpoint

In [ ]:
predictor.delete_endpoint()

## Scaricare il modello in locale

In [35]:
p = estimator.model_dir.split("/")
bucket_name, bucket_path = p[2], p[3]

In [36]:
s3 = boto3.resource('s3')
s3.Bucket(bucket_name).download_file(bucket_path+"/output/model.tar.gz", "model.tar.gz")

## Risorse
 - [Use TensorFlow with the SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/frameworks/tensorflow/using_tf.html)
 - [TensorFlow script mode training and serving
](https://sagemaker-examples.readthedocs.io/en/latest/sagemaker-python-sdk/tensorflow_script_mode_training_and_serving/tensorflow_script_mode_training_and_serving.html)